In [2]:
##

import gym

from stable_baselines import GAIL, SAC
from stable_baselines.gail import ExpertDataset, generate_expert_traj

import gym
import numpy as np

from stable_baselines.common.policies import MlpPolicy
from stable_baselines.common.vec_env import DummyVecEnv
from stable_baselines import PPO2
from stable_baselines import PPO1
# from particle_env import PrticleEnv
from particle_env_continuous import PrticleEnv

import matplotlib.pyplot as plt
%matplotlib inline
from IPython import display

def show_state(env, step=0, info=""):
    plt.figure(3)
    plt.clf()
    plt.imshow(env.render(mode='rgb_array'))
    plt.title("%s" % (info))
    plt.axis('off')

    display.clear_output(wait=True)
    display.display(plt.gcf())





The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



In [3]:
# Generate expert trajectories (train expert)
model = SAC('MlpPolicy', 'Pendulum-v0', verbose=1)
generate_expert_traj(model, 'expert_pendulum', n_timesteps=100, n_episodes=10)



Creating environment from the given name, wrapped in a DummyVecEnv.




/disk1/maze/project/stable-baselines/venv/lib/python3.5/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))




Instructions for updating:
Use keras.layers.flatten instead.
Instructions for updating:
Please use `layer.__call__` method instead.
Instructions for updating:
Use keras.layers.Dense instead.






Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where




actions (2000, 1)
episode_returns (10,)
obs (2000, 3)
episode_starts (2000,)
rewards (2000,)


{'actions': array([[ 0.02505612],
        [ 0.1140542 ],
        [ 0.1616826 ],
        ...,
        [-0.07597148],
        [-0.04526877],
        [-0.0347923 ]], dtype=float32),
 'episode_returns': array([-1327.69327497, -1333.16051096, -1327.20412749, -1677.76058388,
        -1319.58901462, -1275.49044891, -1474.92099452, -1347.91732419,
        -1451.22090411, -1566.78292894]),
 'episode_starts': array([ True, False, False, ..., False, False, False]),
 'obs': array([[ 0.7357745 , -0.6772266 , -0.63633096],
        [ 0.6959809 , -0.7180603 , -1.1404924 ],
        [ 0.63397974, -0.77334964, -1.6619296 ],
        ...,
        [-0.8862086 ,  0.4632864 ,  0.9763855 ],
        [-0.91468143,  0.40417552,  1.3124546 ],
        [-0.9442005 ,  0.32937112,  1.6087959 ]], dtype=float32),
 'rewards': array([-0.59400856, -0.77170444, -1.05787373, ..., -7.17036772,
        -7.60068989, -8.13220787])}

In [4]:
# Load the expert dataset
dataset = ExpertDataset(expert_path='expert_pendulum.npz', traj_limitation=10, verbose=1)

model = GAIL('MlpPolicy', 'Pendulum-v0', dataset, verbose=1)
# Note: in practice, you need to train for 1M steps to have a working policy
model.learn(total_timesteps=1000)
model.save("gail_pendulum")

del model # remove to demonstrate saving and loading



actions (2000, 1)
episode_returns (10,)
rewards (2000,)
episode_starts (2000,)
obs (2000, 3)
Total trajectories: 10
Total transitions: 2000
Average returns: -1410.1740112587809
Std for returns: 123.22552396603196
Creating environment from the given name, wrapped in a DummyVecEnv.




********** Iteration 0 ************
Optimizing Policy...
sampling

done in 0.733 seconds
computegrad

done in 0.116 seconds
conjugate_gradient
      iter residual norm  soln norm
         0    0.00367          0
         1    0.00223    0.00951
         2   0.000255     0.0267
         3   0.000394     0.0854
         4   0.000571      0.235
         5   7.03e-05      0.284
         6   0.000401      0.315
         7   0.000174      0.321
         8   5.97e-06      0.354
         9   9.21e-08      0.436
        10   8.64e-06      0.437
done in 0.131 seconds
Expected: 0.011 Actual: 0.010
Stepsize OK!
vf
done in 0.052 seconds
sampling
done in 0.599 seconds
computegrad
done in 0.002 seconds
conjugate_gradient

In [ ]:
model = GAIL.load("gail_pendulum")

env = gym.make('Pendulum-v0')
obs = env.reset()
while True:
  print(1)
  action, _states = model.predict(obs)
  obs, rewards, dones, info = env.step(action)
#   env.render()
#  show_state(env)
  print(env.render(mode='rgb_array'))

Loading a model without an environment, this model cannot be trained until it has a valid environment.


/disk1/maze/project/stable-baselines/venv/lib/python3.5/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


1
